In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr  4 23:28:32 2022

@author: parkh
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from SIM7 import *

learning_rate = 0.001  
gamma = 1
buffer_limit = 10000
batch_size = 16

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(68,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,6)
        self.number_of_time_list = np.array([1 for x in range(6)])
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        #print(out)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 5)
        else:
            return act
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        return act,act_list
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/FJSP_SIM4.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)
            
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
        s = env.reset()
        done = False
        score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        print(a_list)
        print(a)
        s_prime, r, done = env.step(a)
        print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    return Flow_time, machine_util, util, makespan, score
Flow_time, machine_util, util, makespan, score =main()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)
      
    

--------------------------------------------------
flow time: 1437, util : 0.826, makespan : 197
n_episode: 1, score : -121.0, n_buffer : 96, eps : 8.0%
--------------------------------------------------
flow time: 1449, util : 0.887, makespan : 176
n_episode: 2, score : -72.0, n_buffer : 144, eps : 8.0%
--------------------------------------------------
flow time: 1545, util : 0.831, makespan : 193
n_episode: 3, score : -123.0, n_buffer : 192, eps : 8.0%
--------------------------------------------------
flow time: 1395, util : 0.859, makespan : 207
n_episode: 4, score : -98.0, n_buffer : 240, eps : 8.0%
--------------------------------------------------
flow time: 1447, util : 0.849, makespan : 179
n_episode: 5, score : -108.0, n_buffer : 288, eps : 8.0%
--------------------------------------------------
flow time: 1390, util : 0.869, makespan : 167
n_episode: 6, score : -88.0, n_buffer : 336, eps : 7.9%
--------------------------------------------------
flow time: 1448, util : 0.816

<ipython-input-1-fc3851ac1087>:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 1583, util : 0.810, makespan : 189
n_episode: 20, score : -139.0, n_buffer : 1008, eps : 7.8%
--------------------------------------------------
flow time: 1582, util : 0.844, makespan : 173
n_episode: 21, score : -102.0, n_buffer : 1056, eps : 7.8%
--------------------------------------------------
flow time: 1546, util : 0.843, makespan : 179
n_episode: 22, score : -103.0, n_buffer : 1104, eps : 7.8%
--------------------------------------------------
flow time: 1383, util : 0.868, makespan : 203
n_episode: 23, score : -98.0, n_buffer : 1152, eps : 7.8%
--------------------------------------------------
flow time: 1266, util : 0.843, makespan : 192
n_episode: 24, score : -102.0, n_buffer : 1200, eps : 7.8%
--------------------------------------------------
flow time: 1216, util : 0.836, makespan : 187
n_episode: 25, score : -107.0, n_buffer : 1248, eps : 7.8%
--------------------------------------------------
flow time: 127

--------------------------------------------------
flow time: 1380, util : 0.891, makespan : 184
n_episode: 73, score : -78.0, n_buffer : 3552, eps : 7.3%
--------------------------------------------------
flow time: 1177, util : 0.925, makespan : 172
n_episode: 74, score : -46.0, n_buffer : 3600, eps : 7.3%
--------------------------------------------------
flow time: 1337, util : 0.922, makespan : 172
n_episode: 75, score : -53.0, n_buffer : 3648, eps : 7.3%
--------------------------------------------------
flow time: 1242, util : 0.912, makespan : 186
n_episode: 76, score : -58.0, n_buffer : 3696, eps : 7.2%
--------------------------------------------------
flow time: 1314, util : 0.879, makespan : 188
n_episode: 77, score : -80.0, n_buffer : 3744, eps : 7.2%
--------------------------------------------------
flow time: 1276, util : 0.923, makespan : 163
n_episode: 78, score : -48.0, n_buffer : 3792, eps : 7.2%
--------------------------------------------------
flow time: 1319, ut

--------------------------------------------------
flow time: 1184, util : 0.928, makespan : 169
n_episode: 126, score : -44.0, n_buffer : 6096, eps : 6.7%
--------------------------------------------------
flow time: 1167, util : 0.930, makespan : 164
n_episode: 127, score : -42.0, n_buffer : 6144, eps : 6.7%
--------------------------------------------------
flow time: 1304, util : 0.903, makespan : 183
n_episode: 128, score : -64.0, n_buffer : 6192, eps : 6.7%
--------------------------------------------------
flow time: 1314, util : 0.925, makespan : 180
n_episode: 129, score : -49.0, n_buffer : 6240, eps : 6.7%
--------------------------------------------------
flow time: 1346, util : 0.924, makespan : 185
n_episode: 130, score : -51.0, n_buffer : 6288, eps : 6.7%
--------------------------------------------------
flow time: 1272, util : 0.872, makespan : 178
n_episode: 131, score : -90.0, n_buffer : 6336, eps : 6.7%
--------------------------------------------------
flow time: 12

...

--------------------------------------------------
flow time: 1302, util : 0.911, makespan : 185
n_episode: 762, score : -58.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1184, util : 0.912, makespan : 170
n_episode: 763, score : -55.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1522, util : 0.886, makespan : 181
n_episode: 764, score : -84.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1286, util : 0.884, makespan : 169
n_episode: 765, score : -72.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1295, util : 0.897, makespan : 170
n_episode: 766, score : -67.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1267, util : 0.892, makespan : 184
n_episode: 767, score : -70.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 1279, util : 0.908, makespan : 169
n_episode: 815, score : -58.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1166, util : 0.942, makespan : 160
n_episode: 816, score : -35.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1178, util : 0.945, makespan : 160
n_episode: 817, score : -34.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1097, util : 0.914, makespan : 157
n_episode: 818, score : -52.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1219, util : 0.871, makespan : 182
n_episode: 819, score : -81.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1142, util : 0.913, makespan : 178
n_episode: 820, score : -55.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 1171, util : 0.909, makespan : 183
n_episode: 868, score : -58.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1420, util : 0.868, makespan : 173
n_episode: 869, score : -90.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1282, util : 0.895, makespan : 181
n_episode: 870, score : -71.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1153, util : 0.880, makespan : 192
n_episode: 871, score : -82.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1114, util : 0.931, makespan : 164
n_episode: 872, score : -42.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1108, util : 0.919, makespan : 172
n_episode: 873, score : -49.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 1269, util : 0.900, makespan : 169
n_episode: 921, score : -63.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1359, util : 0.877, makespan : 193
n_episode: 922, score : -84.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1261, util : 0.876, makespan : 187
n_episode: 923, score : -84.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1114, util : 0.931, makespan : 164
n_episode: 924, score : -42.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1125, util : 0.934, makespan : 160
n_episode: 925, score : -41.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1164, util : 0.941, makespan : 163
n_episode: 926, score : -37.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 1121, util : 0.936, makespan : 160
n_episode: 974, score : -39.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1164, util : 0.941, makespan : 163
n_episode: 975, score : -37.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1737, util : 0.847, makespan : 187
n_episode: 976, score : -113.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1280, util : 0.907, makespan : 166
n_episode: 977, score : -61.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1133, util : 0.930, makespan : 165
n_episode: 978, score : -43.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1142, util : 0.913, makespan : 178
n_episode: 979, score : -59.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow t